In [13]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import DraperQFTAdder
from qiskit.visualization import plot_histogram
import numpy as np

# Quantum circuit that computes the absolute difference between two numbers
def absolute_difference_circuit(a, b, n):
    """
    Creates a quantum circuit that computes the absolute difference between two n-bit numbers a and b.
    """
    # Quantum registers for inputs, outputs, and ancillary qubits
    qa = QuantumRegister(n, name='a')
    qb = QuantumRegister(n, name='b')
    qc = QuantumRegister(n + 1, name='c')  # For the result of subtraction (n bits) and overflow
    qabs = QuantumRegister(1, name='abs')  # Qubit to store the sign
    cr = ClassicalRegister(n + 1, name='result')

    # Create the quantum circuit
    circuit = QuantumCircuit(qa, qb, qc, qabs, cr)

    # Initialize the registers with the binary representations of a and b
    for i in range(n):
        if (a >> i) & 1:
            circuit.x(qa[i])
        if (b >> i) & 1:
            circuit.x(qb[i])

    # Subtract b from a and store the result in qc
    # Use quantum subtractor circuit (we'll use the adder with negated b)
    # Negate b by flipping its bits
    for i in range(n):
        circuit.x(qb[i])

    # Add a and ~b (which is -b - 1), so we need to add 1 to complete two's complement
    # We'll use the Draper QFT Adder
    adder = DraperQFTAdder(n)
    circuit.append(adder.to_instruction(), qc[:n] + qb[:n])

    # Add 1 to complete two's complement
    circuit.x(qc[0])

    # Correct the overflow
    circuit.cx(qc[n - 1], qc[n])

    # Compute the sign (if overflow qubit is 1, the result is negative)
    circuit.cx(qc[n], qabs[0])

    # If negative, compute the absolute value by flipping the bits
    for i in range(n):
        circuit.cx(qabs[0], qc[i])

    # Measure the result
    circuit.measure(qc[:n], cr[:n])

    return circuit

# Compute the total distance between two lists using quantum circuits
def total_distance_quantum(L, R):
    # Sort both lists
    L_sorted = sorted(L)
    R_sorted = sorted(R)

    # Ensure the lists are of equal length
    if len(L_sorted) != len(R_sorted):
        raise ValueError("The lists must be of the same length.")

    # Determine the number of bits needed to represent the largest number
    max_num = max(L_sorted + R_sorted)
    n = max_num.bit_length()

    total_distance = 0

    for l_val, r_val in zip(L_sorted, R_sorted):
        # Create the circuit for absolute difference
        circuit = absolute_difference_circuit(l_val, r_val, n)

        # Transpile the circuit for the simulator
        simulator = AerSimulator(method='extended_stabilizer', max_memory_mb=600000)
        transpiled_circuit = transpile(circuit, simulator)
        result = simulator.run(transpiled_circuit, shots=1024).result()
        counts = result.get_counts()
        # Get the most frequent measurement outcome
        measured_value = max(counts, key=counts.get)
        absolute_difference = int(measured_value, 2)

        # Add to the total distance
        total_distance += absolute_difference

    return total_distance

# Example lists (you can replace these with your actual lists)
L = [64430, 87936, 98310, 98142, 73170, 65508, 72642, 63792, 88319, 38149, 88982, 34137, 79589, 49164, 49746, 53896, 21240, 88691, 47697, 38234, 74901, 70760, 16084, 77408, 81054, 68478, 52984, 14952, 42548, 21427, 92508, 22650, 48388, 49540, 65666, 45713, 42419, 64757, 58273, 10576, 31508, 21718, 55862, 29892, 35978, 18953, 51379, 16417, 66023, 28031, 69178, 56463, 33368, 16434, 97551, 46904, 25697, 37363, 11282, 35209, 66763, 85271, 33759, 17008, 97073, 24061, 53095, 12374, 25571, 37178, 88710, 64979, 49734, 73780, 55802, 78442, 97220, 39845, 27359, 66371, 50236, 54937, 11989, 61602, 83376, 84545, 24250, 29247, 48897, 89795, 60748, 32745, 66967, 63073, 71949, 30876, 43640, 37703, 19198, 43605, 35757, 20959, 29302, 56530, 95611, 60818, 18302, 25344, 91669, 67090, 44280, 17423, 56952, 23067, 45925, 84963, 29175, 89724, 85282, 86399, 34615, 47895, 68604, 30689, 66582, 72688, 47389, 66229, 32205, 95885, 56440, 87356, 67641, 20678, 31065, 95705, 90463, 51401, 73186, 45910, 26499, 94815, 46840, 47483, 11870, 28799, 24832, 83929, 39407, 73986, 66614, 75392, 70204, 99686, 85726, 47638, 81669, 70370, 53783, 54371, 72567, 87969, 81067, 61446, 76471, 64431, 42080, 14486, 82716, 49351, 36159, 56913, 96843, 66639, 91239, 65785, 49114, 58894, 45307, 38693, 21963, 46504, 22201, 15556, 86120, 21887, 32242, 61544, 29147, 24239, 30163, 95900, 71487, 69076, 68557, 80423, 92873, 59391, 15416, 90803, 25081, 10323, 75418, 92762, 32178, 72035, 30702, 70262, 10801, 69510, 38295, 57266, 39071, 20540, 57027, 77729, 96980, 65961, 55385, 86139, 72873, 95457, 84209, 93774, 67731, 81176, 50239, 74772, 91982, 40004, 52769, 40693, 50506, 71898, 99210, 45079, 17581, 83211, 52582, 62597, 20716, 18630, 20835, 75099, 56109, 31503, 60392, 26052, 49938, 16604, 94700, 52740, 84337, 46814, 68259, 79927, 57010, 73378, 83131, 84704, 36494, 43734, 85678, 65197, 80121, 97038, 59873, 54928, 46120, 68979, 88690, 38840, 66900, 59182, 15538, 60198, 57452, 12212, 62853, 67754, 86454, 94629, 80986, 76966, 78419, 12343, 26294, 18240, 72971, 69610, 96198, 25389, 29670, 39477, 58028, 77097, 95031, 32844, 12618, 82959, 40438, 58980, 26176, 81464, 31997, 82384, 54010, 63959, 43239, 45375, 36278, 96601, 82935, 13493, 17988, 59523, 69133, 59959, 89068, 31067, 55300, 36759, 91317, 58240, 22368, 60765, 72733, 19688, 90158, 11176, 53051, 76632, 98132, 73098, 46753, 44849, 62899, 71237, 56670, 18624, 32614, 98452, 51626, 55943, 59443, 96628, 77928, 76256, 13944, 12446, 70331, 46097, 46415, 29460, 73685, 53969, 82435, 65769, 89700, 80281, 97402, 11127, 50692, 43989, 40319, 85446, 58820, 77545, 34103, 83858, 23402, 82147, 15719, 16987, 41519, 29178, 92027, 92998, 52103, 64962, 81888, 92252, 88397, 13852, 73603, 95704, 36045, 38834, 85241, 94036, 90984, 44738, 48078, 40832, 78789, 42210, 16967, 76704, 69647, 41412, 50365, 33725, 90837, 25000, 55672, 50507, 82260, 58284, 79462, 76384, 49652, 24386, 76993, 11077, 58255, 81122, 63846, 69361, 19338, 28857, 33643, 61915, 54227, 16162, 21546, 15120, 63185, 44681, 25644, 12963, 41952, 46649, 46412, 32401, 81393, 13678, 68025, 27934, 91794, 98970, 18314, 19908, 69828, 89139, 71402, 64070, 55145, 65851, 98808, 57244, 18541, 88135, 77694, 52608, 67204, 47233, 99317, 84111, 71316, 24604, 28504, 64952, 72718, 55834, 30446, 37415, 41743, 26267, 33658, 50677, 71609, 26047, 31723, 44390, 53788, 84829, 57972, 93241, 38576, 44958, 94256, 41263, 75946, 20333, 55509, 39858, 88032, 98778, 89995, 28456, 61789, 78073, 77073, 86999, 42298, 50995, 32947, 40617, 84633, 19900, 17816, 32530, 17673, 38208, 54880, 10746, 36110, 80179, 55819, 28442, 88289, 83346, 37597, 45978, 52946, 64609, 24593, 92409, 60718, 26498, 14277, 90961, 19915, 68301, 56321, 15922, 80444, 49913, 97519, 81867, 72049, 98003, 29774, 83596, 65711, 94799, 52228, 35681, 86161, 99764, 13342, 19557, 85701, 67265, 24579, 99212, 11279, 81879, 42791, 98914, 29879, 87696, 54212, 30014, 43433, 31827, 27323, 47887, 12778, 38504, 11985, 88281, 60313, 84487, 10753, 11349, 58532, 37463, 74158, 47713, 52617, 75642, 86070, 89264, 13839, 16425, 15973, 97324, 74858, 30114, 91599, 47093, 50285, 14846, 89741, 27133, 38409, 34370, 43982, 22858, 57345, 40807, 75561, 14111, 49442, 61122, 97636, 43772, 55030, 52741, 74578, 50656, 62586, 99380, 41885, 95691, 68929, 32180, 27880, 15658, 40225, 54028, 51248, 55585, 34730, 72713, 48744, 50628, 66099, 84471, 74726, 63777, 38782, 62565, 30082, 66224, 17798, 65148, 50731, 60010, 71057, 79087, 63666, 61860, 46372, 40076, 81205, 29232, 91161, 80675, 45465, 80432, 84712, 32924, 27386, 35781, 28248, 33634, 93878, 26858, 63868, 94063, 26657, 65635, 47508, 87138, 33328, 70031, 81151, 42263, 85424, 21504, 76646, 36770, 76459, 58215, 20418, 68543, 71082, 24484, 70941, 71613, 98938, 42671, 11798, 55015, 92532, 10415, 89920, 36610, 11196, 61243, 47794, 10054, 90743, 12980, 43206, 23376, 45693, 79629, 69524, 30811, 76688, 50584, 46173, 22138, 98933, 55451, 97723, 39097, 29080, 58962, 71265, 14559, 14366, 68344, 48432, 38005, 85989, 98422, 20685, 69814, 11356, 39329, 52898, 43135, 20409, 52446, 63515, 19371, 16430, 45803, 66092, 80760, 64945, 43951, 87271, 91354, 32506, 15452, 87783, 90937, 68208, 91228, 35773, 74252, 96573, 29447, 61206, 75242, 76850, 22095, 78467, 27611, 18974, 74621, 19469, 26785, 99803, 96042, 61772, 21746, 44817, 65817, 90360, 40529, 63336, 57489, 41980, 61947, 72393, 72994, 38393, 61264, 28725, 68461, 68050, 97146, 13195, 48395, 55269, 43616, 76357, 50496, 97083, 32806, 71713, 91635, 23121, 27917, 61595, 96038, 51300, 99332, 49914, 87083, 39253, 98686, 63961, 76429, 84657, 18867, 28534, 20843, 40011, 28716, 64363, 41215, 74285, 37245, 53602, 73935, 35283, 84164, 26264, 49434, 67281, 47315, 21216, 57730, 55530, 23807, 79378, 81270, 89580, 81287, 37987, 29721, 76282, 82973, 43095, 57911, 15665, 97099, 77440, 61564, 38697, 91585, 66058, 96221, 80464, 52031, 74599, 44495, 37781, 42333, 25393, 31875, 25513, 52578, 83883, 13961, 86512, 52957, 70656, 32267, 67751, 83650, 11249, 82052, 60178, 78052, 92063, 47149, 82429, 24749, 26862, 77843, 24471, 15084, 15444, 15492, 22651, 62625, 49448, 47125, 45650, 76382, 28236, 24712, 26908, 51661, 33112, 59018, 12648, 47726, 16833, 17720, 64016, 62900, 66411, 47653, 78845, 98508, 83446, 50837, 85485, 41709, 84178, 27821, 89199, 95089, 80677, 84551, 84718, 14520, 61473, 69357, 36592, 88660, 92407, 17708, 27217, 51540, 93978, 83790, 46160, 21508, 20593, 12368, 46760, 46693, 90007, 63689, 98781, 77580, 24571, 37950, 63333, 14765, 41979, 65602, 34240, 40909, 25236, 56096, 85735, 37637, 34344, 53389, 66490, 18024, 45153, 94052, 67564, 54685, 89848, 15698, 67033, 23435, 99883, 32300, 82028, 19382, 40073, 86881, 42474, 59165, 60370, 74295, 98654, 87697, 68417, 32125, 23153, 32963, 18366, 74792, 78281, 41693, 41188, 19719, 44395, 75495, 14585, 94358, 13464, 58483, 44941, 91349, 95485, 58981, 43473, 51945, 40385, 32646, 73150, 12154, 55441, 98025, 15339, 31543, 65949, 31434, 57312, 91833, 23286, 58987, 12591, 37648, 97542, 33381, 69851, 60076, 86697, 25533, 68777, 47390, 94550, 61539]

R = [75582, 20843, 72035, 69076, 50561, 20716, 84718, 89556, 66913, 43160, 10253, 95932, 92389, 98263, 92762, 43420, 95725, 37648, 66763, 98154, 20452, 11404, 74024, 32178, 45709, 92252, 55239, 20843, 69422, 20329, 81533, 76459, 81843, 25697, 12373, 16199, 10468, 30375, 43473, 20540, 42210, 52578, 92252, 20716, 50547, 43473, 31038, 83883, 24679, 55268, 45771, 93602, 20147, 43473, 73418, 19198, 60996, 35781, 92252, 72035, 27103, 69322, 94633, 83883, 21870, 55053, 92252, 92762, 51808, 10953, 67938, 48065, 80760, 81067, 37177, 90493, 32178, 25697, 39117, 17473, 82028, 41865, 69107, 78467, 75395, 42210, 55446, 37936, 21653, 26960, 39097, 20716, 43473, 21718, 83590, 83883, 21718, 37133, 43473, 42609, 54910, 23081, 52984, 82308, 42671, 39516, 61539, 25697, 80760, 98098, 59977, 65400, 78467, 72035, 76133, 87050, 32178, 74826, 49648, 77097, 52896, 52578, 72035, 20716, 92521, 76209, 87846, 64769, 70746, 83814, 19941, 21436, 92252, 45960, 75217, 69076, 44022, 36472, 48543, 49551, 84718, 20415, 43473, 94710, 43473, 14080, 79589, 51875, 53253, 62364, 44738, 21277, 55613, 21718, 19198, 92252, 30026, 60748, 11004, 35046, 87098, 32178, 60748, 98023, 92252, 25697, 80760, 13466, 52984, 15539, 15090, 88064, 44738, 89714, 42210, 76459, 86785, 41438, 79589, 80286, 52722, 44738, 84718, 61539, 86377, 81067, 92762, 37648, 73990, 62852, 48652, 83883, 60590, 47403, 70174, 85726, 10245, 87955, 80760, 70939, 81067, 46564, 60748, 43409, 20716, 25025, 11797, 20843, 60970, 56933, 27487, 71215, 92542, 21718, 81151, 90745, 28442, 19198, 82179, 91952, 61041, 44738, 83883, 92762, 14982, 26473, 13336, 81067, 80690, 89466, 72035, 42210, 26856, 68597, 32178, 53505, 94678, 28309, 50257, 92762, 38593, 73077, 30809, 18088, 42210, 72884, 83711, 17008, 20540, 85679, 29256, 69859, 25174, 64723, 34542, 18036, 78467, 90472, 30214, 37648, 40475, 90827, 61539, 81067, 24363, 85579, 72035, 17827, 33920, 32133, 90929, 92762, 67817, 85839, 81151, 54586, 71692, 79875, 67888, 53532, 52984, 92403, 30351, 77097, 92762, 20540, 75804, 23339, 58631, 81067, 31081, 13525, 76459, 79589, 21718, 27842, 78097, 43989, 96437, 43688, 20716, 80760, 39610, 24846, 35781, 83883, 20844, 45452, 16738, 87524, 95715, 80760, 27550, 58808, 92762, 67650, 44178, 77097, 20843, 36430, 23059, 48277, 46947, 85726, 92356, 84751, 26888, 62073, 57933, 28442, 50769, 97012, 80760, 83281, 25697, 15301, 74187, 81067, 20649, 82808, 34732, 76464, 35781, 89986, 72035, 73132, 45659, 74161, 94048, 20843, 46956, 39097, 83883, 44307, 89046, 52578, 40606, 45167, 50621, 43473, 60748, 87361, 60748, 61539, 11156, 71698, 89274, 80760, 37577, 33573, 61991, 48488, 80760, 22112, 29897, 47932, 25765, 92252, 77097, 32490, 29503, 57018, 32178, 89473, 20540, 32178, 46227, 72035, 60748, 77097, 92762, 44738, 76459, 24578, 78634, 92252, 61124, 65696, 51055, 35781, 63580, 79129, 17116, 52578, 81986, 73820, 92762, 33808, 43473, 23808, 81151, 44424, 77003, 79166, 43047, 28757, 30206, 76459, 35781, 37648, 89353, 52984, 18855, 92252, 60184, 61012, 68250, 89512, 80760, 42210, 17962, 89796, 68405, 61539, 20540, 78467, 99213, 35781, 61539, 71396, 69562, 97620, 60768, 36928, 25697, 32178, 38138, 25697, 67787, 21718, 79589, 84731, 61539, 83313, 91715, 39097, 37648, 30506, 39097, 61539, 58867, 35781, 43275, 44738, 92252, 29159, 78467, 59675, 11262, 44738, 78322, 52977, 21718, 72035, 89828, 60748, 32178, 27703, 20540, 84718, 47639, 85109, 67054, 77097, 95680, 40117, 21718, 76619, 18411, 62071, 72035, 70754, 31065, 21718, 40782, 43473, 20843, 22994, 40716, 28627, 68538, 25156, 77097, 77097, 78467, 36942, 23213, 61360, 35781, 79589, 92066, 28171, 74543, 84718, 28442, 92762, 39097, 92252, 60223, 70229, 35781, 79589, 79965, 14227, 80760, 67051, 72035, 39097, 83883, 76459, 32178, 81151, 84718, 92936, 41869, 72035, 74833, 15295, 47398, 44738, 66763, 41667, 24392, 54679, 60748, 72720, 20540, 36662, 12924, 75630, 52984, 69076, 32065, 76459, 64434, 43749, 32614, 78467, 90971, 25697, 12152, 32178, 18405, 44738, 76459, 59672, 69076, 81151, 17588, 16319, 99026, 54241, 84718, 68926, 84718, 22018, 35781, 39267, 93659, 82028, 92765, 37648, 60748, 15314, 83883, 42151, 39097, 60748, 69423, 20843, 34197, 61764, 28363, 34713, 81067, 53151, 25131, 25697, 83358, 23263, 81559, 34834, 23884, 73924, 46691, 23340, 18409, 25515, 69609, 25697, 76459, 95026, 83206, 44495, 36987, 43473, 43473, 44738, 81067, 82826, 43646, 80760, 10687, 32178, 36088, 25697, 39097, 37648, 66803, 16037, 20540, 47464, 37648, 74023, 33274, 97575, 25697, 48419, 25173, 51474, 75816, 80760, 73906, 96322, 25697, 80760, 77822, 48276, 48409, 59450, 42103, 13352, 52984, 92762, 11445, 83597, 62612, 85726, 23518, 32662, 71871, 72035, 90060, 31991, 43473, 44495, 86232, 81067, 52984, 86494, 98749, 24014, 35781, 25697, 25697, 81601, 78131, 43260, 11599, 30402, 46252, 80760, 15435, 40848, 52578, 92252, 92762, 69724, 77097, 70562, 20540, 48749, 51485, 77097, 35918, 92762, 52984, 42210, 34368, 60748, 78467, 78467, 13782, 20716, 20540, 72035, 88048, 43473, 93533, 83883, 80045, 72035, 28819, 20843, 19198, 25697, 25697, 34899, 84718, 20540, 20540, 74304, 33425, 95646, 37648, 78467, 17766, 75924, 37648, 79052, 42991, 75439, 97868, 92252, 92762, 32178, 74280, 31544, 62085, 75894, 77838, 24247, 55285, 20540, 84363, 83883, 19323, 25697, 77726, 21449, 39950, 79309, 14075, 54251, 84880, 83883, 62916, 39191, 76639, 69076, 92252, 20540, 42671, 25490, 27770, 83883, 19198, 44738, 32178, 14399, 12291, 62312, 19177, 55565, 24002, 69076, 47157, 92762, 20540, 92762, 77858, 78467, 30664, 39795, 33272, 54823, 99039, 23548, 20540, 42671, 78621, 58861, 92703, 72035, 42210, 35781, 94627, 39358, 77374, 20843, 35781, 90042, 25697, 92252, 19198, 92252, 76459, 37648, 70552, 44738, 83883, 60687, 65500, 19442, 78935, 76459, 82563, 71638, 93445, 44738, 76459, 42481, 30034, 20843, 61539, 40937, 50046, 71537, 44507, 92762, 31623, 88911, 81151, 81151, 50336, 92252, 37648, 40879, 60748, 20540, 67476, 80760, 72429, 37648, 20843, 78467, 23702, 76459, 44738, 44240, 19059, 80760, 42210, 42671, 61080, 68627, 32614, 56271, 41563, 62867, 43417, 61539, 80760, 21763, 48757, 36097, 21134, 40592, 81067, 28270, 63823, 92252, 54575, 52984, 35781, 76268, 15099, 24491, 31467, 37648, 97080, 32178, 16864, 26852, 10379, 40936, 63694, 37648, 28689, 57472, 43473, 43852, 20716, 20125, 11853, 20843, 92762, 37320, 24917, 35781, 20540, 20540, 12607, 44495, 77706, 34181, 69737, 54282, 84718, 38902, 16081, 75780, 89913, 76234, 92745, 44738, 39242, 98910, 76587, 74779, 55432, 13148, 32614, 22506, 78757, 32614, 39877, 52969, 33132, 47303, 36750, 48801, 91679, 20540, 74190, 84718, 69076, 44738, 35486, 24004, 69819, 68680, 21625, 78452, 37648, 31642, 41279, 80760, 34541, 73389, 42210, 39399, 55320, 87304, 51288, 12352, 57673, 50327, 43473, 49359, 35781, 72035, 72831, 78467, 21023, 23804, 23423, 85726, 69076, 12344, 83883, 65861, 61539, 69762, 44738, 42174, 81067, 37648, 20843, 39097, 38796, 42210, 42245, 32614, 57203, 76885, 52984, 32178, 37648, 32316, 35781, 44140, 92540, 19198, 32614, 96431, 15175, 75651, 80760, 20843]

# Compute the total distance
distance = total_distance_quantum(L, R)
print(f"The total distance: {distance}")


QiskitError: 'ERROR:  [Experiment 0] a circuit requires more memory than max_memory_mb. ,  ERROR: a circuit requires more memory than max_memory_mb.'